In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import glob

When I do the electrochemical measurement I take readings for OCP, EIS, CV,LSV_dark, LSV_light

Except EIS and OCP for all the above mentioned measurements I need to plot EvsRHE in X axis and Current density in Y axis.

The equations are as follows-

X axis-E vs RHE = Col("Potential")+Ered.+(0.059*pH)
where Ered is the reduction potential of the reference electrode. If it is Calomel it is "0.280", MOE it is "0.098" 
Y axis- Current density = Col("Current")/0.1979


EIS I need to figure out a little and then I will tell you. But for the rest I will need help from you to write a code where the code fetches the data and then takes the desired column, calculates and plots the data for me.


SAMPLE NAMING

D3531_Co2Au1_ Si_RCA850_2h_300C_O2_KOH1M_Aucontact_amel
DEPOSITION NO_SAMPLEDEPOSITED_SUBSTRATE_CONDITION parameterA_CONDITION parameter B_CONDITION parameter C_CONDITION parameter D_CONTACT_Reference electrode

In [3]:
main_path = "/Users/anirudh/Python/pcepc_properties/20250327" # str(input("Enter the main path"))
folders = os.listdir(main_path)
sub_dirs = []

In [ ]:
def calculate_xy(file, E_red, pH):
    df = pd.read_csv(file, sep = "\\s+", skiprows = 19, header = None)
    df.columns = ["No", "time", "potential", "current"]

    x = df["potential"] + E_red + (0.059 * pH) #Col("Potential")+Ered.+(0.059*pH)
    y = df["current"]/0.1979 #Col("Current")*1000
    return x, y

# x,y = calculate_xy("/Users/anirudh/Python/pcepc_properties/20250327/D3386_FeSn_50nm_550C_5h_KOH1M_AMEL/d3386_fesn_50nm_550c_5h_koh1m_amel_cvdark_sol.txt",1,1)

In [ ]:
lsv_master_dict = {}
cv_master_dict = {}

master_df_list = []

for f in folders:
    E_red, pH = 0.28, 13.8#float(input("Enter the E_red value (Enter 0 if it is not needed for your calculation): ")), float(input("Enter the pH value (Enter 0 if it is not needed for your calculation): "))
    lsv_files = glob.glob(os.path.join(main_path, f, "*lsv*txt"))
    cv_files = glob.glob(os.path.join(main_path, f, "*cv*txt"))

    for lsv_file in lsv_files:
        x,y = calculate_xy(lsv_file, E_red, pH)

        plt.plot(x, y, label=lsv_file.split('/')[-1].split('.')[0])
        
        plt.xlabel('E vs RHE (V)')
        plt.ylabel('Current Density')
        plt.legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize=8)
        
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(lsv_file.replace('.txt', '_output.csv'), index=False)

        df['Folder'] = f
        df['File'] = os.path.basename(lsv_file)
        df['Type'] = 'LSV'
        master_df_list.append(df)

    plt.grid()
    plt.savefig(os.path.join(main_path, f, "lsv_plot.png"), dpi=300)
    plt.savefig(os.path.join(main_path, f, "lsv_plot.pdf"), dpi=300)
    plt.show()


    for cv_file in cv_files:
        x,y = calculate_xy(cv_file, E_red, pH)

        plt.plot(x, y, label=cv_file.split('/')[-1].split('.')[0])
        
        plt.xlabel('E vs RHE (V)')
        plt.ylabel('Current Density')
        plt.legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize=8)
        
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(cv_file.replace('.txt', '_output.csv'), index=False)

                # Append to master
        df['Folder'] = f
        df['File'] = os.path.basename(cv_file)
        df['Type'] = 'CV'
        master_df_list.append(df)


    plt.grid()
    plt.savefig(os.path.join(main_path, f, "cv_plot.png"), dpi=300)
    plt.savefig(os.path.join(main_path, f, "cv_plot.pdf"), dpi=300)
    plt.show()
    

master_df = pd.concat(master_df_list, ignore_index=True)
master_df.to_csv(os.path.join(main_path, "master_output.csv"), index=False)

lsv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in lsv_master_dict.items()]))
cv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in cv_master_dict.items()]))

lsv_master_df.to_csv(os.path.join(main_path, "master_LSV.csv"), index=False)
cv_master_df.to_csv(os.path.join(main_path, "master_CV.csv"), index=False)


In [5]:
lsv_master_dict = {}
cv_master_dict = {}

for f in folders:
    E_red, pH = 0.28, 13.8
    lsv_files = glob.glob(os.path.join(main_path, f, "*lsv*txt"))
    cv_files = glob.glob(os.path.join(main_path, f, "*cv*txt"))

    for lsv_file in lsv_files:
        x, y = calculate_xy(lsv_file, E_red, pH)

        # Save individual output
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(lsv_file.replace('.txt', '_output.csv'), index=False)

        # Add to master dict
        key_base = os.path.basename(lsv_file).replace('.txt', '')
        lsv_master_dict[f"{key_base}_x"] = x
        lsv_master_dict[f"{key_base}_y"] = y

    for cv_file in cv_files:
        x, y = calculate_xy(cv_file, E_red, pH)

        # Save individual output
        df = pd.DataFrame({'E_vs_RHE': x, 'Current_density': y})
        df.to_csv(cv_file.replace('.txt', '_output.csv'), index=False)

        # Add to master dict
        key_base = os.path.basename(cv_file).replace('.txt', '')
        cv_master_dict[f"{key_base}_x"] = x
        cv_master_dict[f"{key_base}_y"] = y

# Convert to DataFrames and save
lsv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in lsv_master_dict.items()]))
cv_master_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in cv_master_dict.items()]))

lsv_master_df.to_csv(os.path.join(main_path, "master_LSV.csv"), index=False)
cv_master_df.to_csv(os.path.join(main_path, "master_CV.csv"), index=False)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 13, saw 9
